# DeepSeaAI
#### Pipeline used for the cleaning and analysis of citizen science data with AI.

This markdown will explain how to clean your citizen science data, and train a Yolov8 model on it. It has been developed with the intent to deal with DeepSeaSpy data (accessible from : https://ocean-spy.ifremer.fr/). 

Note : some functions may copy your images (specifically **vision**, **catalog**, **prepare_yolo**). Please be wary of the space available on your computer and act accordingly.

### Contents
You should have 2 python files and a Jupyter notebook file (download here):

```
DeepSeaAI
├── Functions.py            <- Functions used for the cleaning/analysis.
├── DeepSeaLab.ipynb        <- Here
```

**Functions** contains all the functions needed for the cleaning and analysis of citizen science datasets. Detailed explanations of the functions are to be found here. You can modify them and use them as the basis for your work. There are also functions that are useful just for the exploration of your dataset.

**DeepSeaLab** is a ready-to-use file that you can change based on your needs/dataset. There are also examples on how to use the pipeline.

In [ ]:
import sys

print("Python version:", sys.version_info)

### Requirements
Import the necessaries functions for the cleaning/analysis

In [ ]:
import os, csv, json, collections, random, shutil
os.chdir(r'C:\Users\alebeaud\Desktop\Save\DeepSeaAI\EN\Python Files')
import pandas as pd
from pathlib import Path
from Functions import polygones2bb, points2bb, lines2bb, convert_yolo, prepare_yolo, vision, SaveCSV, create_yaml
from Functions import unite, catalog, get_df
import matplotlib.pyplot as plt
os.getcwd()

### Access to data/files
3 main paths are to be defined :

```
path_csv
```
Location of your dataset.


```
path_img
```
Path to the folder where your images are stored.
the format of the images should be in .jpg

```
images
├── image 1.jpg           
├── image 2.jpg          
├── ... 
```

Then, finally
```
path_save
```
Where to store the cleaned dataset, catalogs, etc...

In [ ]:
# csv access :
path_csv=
# images :
path_img=Path() 
# save
path_save=

## Conversion of bounding boxes

Our pipeline can convert 3 type of bounding boxes into regular bounding boxes.
If your data already satisfies the following format, you can skip this part.

|xmin |ymin |xmax |ymax |
|-----|-----|-----|-----|
|972  |982  |549  |559  |


#### Polygons
On DeepSeaSpy, polygons are in the json format :

```
[{\x\":282,\"y\":115},{\"x\":15,\"y\":538},{\"x\":50,\"y\":679},{\"x\":285,\"y\":497}]
```

The column containing the polygon values can be named :
|polygon_values                                                                         |
|---------------------------------------------------------------------------------------|
|[{\x\":282,\"y\":115},{\"x\":15,\"y\":538},{\"x\":50,\"y\":679},{\"x\":285,\"y\":497}] |

Therefore our pipeline was made to deal with such format. If you need to convert your own type of polygons, you can modify the way points are stored in the **polygons2bb** function.

#### Lines
Lines are to be in the following format, with two points defined by (x1,y1) and (x2,y2).

|x1 |y1 |x2 |y2 |length|
|---|---|---|---|------|
|761|451|859|364|131   |

Length is used to correct the converted bounding box, depending on the line's angle with the x axis. If the line is too vertical or too horizontal, the lines2bb function automatically corrects the converted bounding box. By default, if the angle is of +-5 degrees, the corrections happens. You can modify/find mor info in the Functions.py file.

#### Points
You can manually set a padding on the x and y axis in the Functions.py file.

|x1 |y1 |
|---|---|
|761|451|

The padding is the same for every point in your dataset, if you wish to use a different one for different species/uses, we recommend you split your dataset and run each part with a different padding. Then, you can concatenate all of your subsets with :

```
pd.concat([polybb,lignesbb,pointsbb])
```

We alsor recommend changing the names of your images columns and species columns, so that our functions can run properly.

|name_img         |name_sp     |
|-----------------|------------|
|'MOMAR_90095.jpg'|'Buccinidae'|

In [ ]:
# Import your dataset
data=
data.head

In [ ]:
# Rename data from DeepSeaSpy
#data.rename(columns={'pos1x': 'x1', 'pos1y': 'y1','pos2x': 'x2', 'pos2y': 'y2','name_fr':'name_sp','name':'name_img'}, inplace=True)

# Subset of polygon labels
poly=
# Subset of lines labels
lines=
# Subset of points labels
points=

# Polygons
polybb=polygones2bb(poly)
# Lines
lignesbb=lines2bb(lines)
# Points
pointsbb=points2bb(points)

# Concatenate your split dataset into a single one
bb=pd.concat([polybb,lignesbb,pointsbb])

In [ ]:
# Save the converted dataset
SaveCSV(bb,path_save,'export_bb')

In [ ]:
bb=pd.read_csv(os.path.join(path_save,'export_bb'), sep=None, engine='python')

## Vision

We encourage you to use the vision function, which will allow you to visualize your images with your bounding boxes added onto them.
First, you have to define the object 'colors', which is a dictonary containing each species with its corresponding color coded in BGR.

If you wish to, you can limit the number of saved images by adding 'nb_img' as an argument. 
```
vision(bb,colors,path_img,path_save=None,nb_img=None)
```
When not specified, it saves the images in the parent directory of the path_img.

This function copies the images from path_img, so it may generate a lot of data if you don't specify a number of images.

In [ ]:
# Colors are in BGR
couleurs = {
        'Escargot buccinidé': (0, 0, 255),  # Red
        'Couverture de moules': (255, 0, 0),  # Blue
        'Couverture microbienne': (0, 255, 0),  # Green
        'Couverture vers tubicole': (255, 255, 0),  
        'Crabe araignée': (128, 0, 128),  
        'Crabe bythograeidé': (255, 165, 0),  
        'Crevette alvinocarididae': (255, 192, 203),  
        'Escargot buccinidé': (0, 0, 255),  
        'Ophiure': (139, 69, 19),  
        'Poisson Cataetyx': (0, 255, 255),  
        'Poisson chimère': (0, 0, 0),  
        'Poisson zoarcidé': (192, 192, 192),  
        'Pycnogonide': (255, 215, 0),  
        'Ver polynoidé': (255, 255, 255),  
        'Vers polynoidés': (245, 245, 220),
        'Autre poisson':(245, 245, 220)
    } 

vision(bb, couleurs, path_img, path_save, nb_img=10) #nb_img is optional

## Unification of overlapping bounding boxes

This step ensures that there is no redundancy in your dataset. You can skip this part if you are not dealing with this kind of problem.

The unification of the bounding boxes is done when they are strictly overlapping (while the iou value is kept as None).
Still, if you wish to limit the unification of the BB to a certain superposition threshold (iou), you can.

```
unite(dataframe, iou=None, grouper_0=False)
```

iou_thresh corresponds to the minimum Intersection over Union (IoU) value between two bounding boxes to consider them overlapping.

Bounding boxes that are not overlapping any are automatically discarded. If you want to keep them, you can change the argument grouper_0 to grouper_0=True.

The function keeps track of how many bounding boxes the final ones are made of in the column "occurrences".



In [ ]:
# Unification of all overlapping bounding boxes
ubb=unite(bb)

In [ ]:
# Keeping bounding boxes only if they are made of at least 3 overlapping ones
ubb=ubb[ubb['occurences']>=3]

In [ ]:
# Unification only when bounding boxes are 0.2% overlapping
ubb=unite(bb,0.2) 

In [ ]:
# Unification all of your bounding boxes, and not discarding isolated ones
ubb=unite(bb,grouper_0=True) 

In [ ]:
# Save your dataframe
SaveCSV(ubb,path_save,'ubb')

In [ ]:
ubb=pd.read_csv(r'C:\Users\alebeaud\Desktop\save_notebook\ubb.csv', sep=',')

In [ ]:
import os, csv, json, collections, random, shutil
import pandas as pd
from pathlib import Path
from Functions import polygones2bb, points2bb, lines2bb, convert_yolo, prepare_yolo, vision, SaveCSV, create_yaml
from PIL import Image
from Functions import unite, catalog, get_df
import matplotlib.pyplot as plt
os.getcwd()
ubb=pd.read_csv(r'C:\Users\alebeaud\Desktop\save_notebook\ubb.csv', sep=',')
# csv access :
path_csv=r'Q:/export_dss_20191018_Clean.csv'
# images :
path_img=Path(r'C:\Users\alebeaud\Desktop\Image_dsp') 
# save
path_save=r'C:\Users\alebeaud\Desktop\save_notebook'

## Catalog

Unite only unifies overlapping bounding boxes, it does not verify if the object you want to labelise is in fact inside the bounding box. If you wish to be very wary about which bounding box are to be kept in your dataset, you can use the two functions :

```
catalog(df, path_img, path_save=None)
```
When not specified, it saves the images in the parent directory of the path_img.

Creates a catalog of snapshots from all the bounding boxes you have in your dataframe (df). You can then delete the snapshots of bounding boxes you want to discard.

```
get_df(df,path_save)
```

From the path_save (where your remaining snapshots are), and your unified dataframe (df), this function returns a dataframe that lists all the remaining bounding boxes from your own cleaning.

In [ ]:
#Create snapshots from images and the dataframe
catalog(ubb, path_img, path_save)

In [ ]:
#After discarding snapshots, get the remaining rows
df=get_df(ubb,path_save)
df.head()

## Prepare for Yolo

You can then use prepare_yolo to split your dataset in 3 (train,val,test), and train yolov8 on it.

```
prepare_yolo(df,path_save,path_img,prop=[.8,.1])
```

The **prop** parameter stands for proportion. It asks for the size of 2 subsets (in order) : **train** and **validation**. The remaining percentage is the size of the **test** subset. The test subset is not mandatory for the training of a model.
prop=[.8,.1] means that the training subsets is 80% of our dataset, the validation subset is 10%. The remaining percentage is the test subset's size, here, 10%.

Yolov8 takes bounding boxes in the following format :

```
class x y w h
```

With x and y the coordinates to the center of the bounding box. W and h are the width and height of the bounding box. Those 4 values are normalised between 0 and 1. prepare_yolo generates 1 txt file for each image, and each line in this file is the description of 1 bounding box.

example :
```
7 0.5713542 0.6847222 0.0359375 0.0731481
```


In [ ]:
#Size of your images
size=[1920,1080]

convert_yolo(df,size)
prepare_yolo(df,path_save,path_img,prop=[.8,.1])

## Yolo training
Now you can train a yolo model with your dataset.

Your yolo training folder path should look like this :

```
yolo_training
├── images        <- Where your images are
|   ├── train
|   ├── val
|   ├── test
├── labels        <- Where your bounding boxes/labels for each image are
|   ├── train
|   ├── val
|   ├── test
```
Yolo needs a yaml file to understands where the data is, and what the classes are.
You can create the file yourself or use create_yaml.

In [ ]:
# Creates a yaml file containing all of the information necessary for running Yolov8 on your data
create_yaml(path_save,df,'output')

Everything is ready for Yolov8.


If you installed our requirements correctly, you can train your Yolov8 model in python :

In [ ]:
from ultralytics import YOLO

In [ ]:
# Load a new YOLO model from scratch
model = YOLO('yolov8n.yaml')  # build a new model from YAML

In [ ]:
# Or load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

In [ ]:
# Get where the .yaml file is stored
output=str('output'+'.yaml')
yaml_path=os.path.join(path_save,output)

# Train the model
results = model.train(data=yaml_path, epochs=10, imgsz=640)

In [ ]:
# Run the model to identify objects
# Load your trained YOLOv8n model
model = YOLO('your/path/to/trained/model/here')
img_to_predict=r'C:\Users\alebeaud\Desktop\images_to_be_predicted'

In [ ]:
list_img=list(img_to_predict.glob('**/*.jpg'))

# Run inference on your list of images
results=model(list_img)

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

You can also launch the training within a command terminal (same line of code for windows or linux). This can be useful if you don't want to open a python interactive window, or you are working remotly. 

In [ ]:
# For the training
!yolo task=detect mode=train model=yolov8n.yaml imgsz=640 data=absolute/path/to/data_yolo.yaml show_labels=False epochs=10 batch=8 name=run1

# Once your model is trained, you can run 'predict' to detect objects
!yolo predict model=path/to/yolo/runs/detect/run1/weights/best.pt source=path/to/data show_labels=False